# IMPORTS

In [ ]:
# imports 

import pandas as pd
import numpy as np

import scipy
import scipy.stats as ss
from scipy.stats import pareto

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import matplotlib.ticker as ticker
from matplotlib.ticker import PercentFormatter
from matplotlib.ticker import FormatStrFormatter

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from collections import Counter

import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

# Graph parameters
- figure out the purpose of ax and plt show or fig etc.
    - do I need to take a course on how to use matplotlib from kaggle or something

In [ ]:
# control size of each 
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)

# whitespace controller
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

# Read in csvs 
- what was that glob thing i did for research project?

In [ ]:
df = pd.read_csv("df.csv")

display(df.info(memory_usage="deep"), df.head())

# Functions

In [ ]:
# see a column
accident_2017['accident_reference']

# see multiple columns
accident_2017[['accident_index', 'first_road_class', 'first_road_number', 'second_road_class', 'first_road_number']]

# count values in a column
accident_2017['accident_reference'].value_counts() 

# get range of dates into a new dataframe, copy to create a fresh instance of the dataset that isnt a slice 
accident_2005_2021 = accident_79_21[accident_79_21.accident_year.isin(range(2005,2022))].copy()

# search a df to return a row
accident_severe_05_20 = accident_2005_2021[accident_2005_2021.accident_severity == 3]

# group by a column and sum then rename the count column
date_counts = accident_2005_2021.groupby(['date'])['date'].count().reset_index(name='counts')

# group by year, then group by month and then count month occurences to get counts on each month of every year
month_counts = accident_2005_2021.groupby(['month','year'])['month'].count().reset_index(name='counts')

# shape of df 
severity_3.shape

# split date into months etc.
accident_2005_2021['day'] = accident_2005_2021['date'].dt.day
accident_2005_2021['month'] = accident_2005_2021['date'].dt.month
accident_2005_2021['year'] = accident_2005_2021['date'].dt.year

# read pkl
accident_79_21 = pd.read_pickle('accident_79_21.pkl')

# concatenate dataframes
accidents_concat = pd.concat([accident_2017, accident_2018, accident_2019], ignore_index=True)

# convert column to a type
accidents_concat['accident_index'] = accidents_concat['accident_index'].astype(str)

# explore the null locations for a variable, see what cols it impacts 
accidents_concat[['accident_index','accident_severity','number_of_casualties','longitude']].query("longitude != longitude").value_counts('accident_severity')

# drop columns
accidents_concat = accidents_concat.drop(columns=['junction_control', 'second_road_class', 'second_road_number'])

# query ultra powerful I like this a lot
accidents_concat.query("accident_index == 2017010001708")

# dataframe join
df3 = pd.merge(df1, df2, how='left', left_on ='accident_index', right_on ='accident_index')

# view nulls 
main_accident_set.isnull().sum(axis = 0)








# bar plots
- sns colour schemes

In [ ]:
# sns countplot like a bar chart

ax = sns.countplot(x="accident_year", 
                   data=accident_2005_2021
                  )
plt.xticks(rotation=45)

In [ ]:
# matplotlib bar chart

accident_2005_2021['accident_year'].value_counts(sort = False).plot(kind='bar')

# alternative to retain index order e.g. years or months
accident_severe_1_05_20['accident_year'].value_counts().sort_index().plot(kind='bar')

plt.xlabel("Year")
plt.ylabel("Count of Accidents")
plt.title("Count of Road Accidents per Year")
plt.xticks(rotation=45)


# data cleaning

In [ ]:
# format the date if its a string
accident_2005_2021['date'] = pd.to_datetime(accident_2005_2021['date'], format = '%d/%m/%Y')

# handle nulls,  - careful need to check again
accidents_concat.replace(-1, np.NaN, inplace=True)
accidents_concat.isnull().sum(axis = 0)

# Time series analysis 

In [ ]:
# trends through the year, counts on each day plotted out 

plt.figure(figsize=(30,8))
sns.lineplot(data=date_counts, 
             x='date', 
             y='counts');

plt.savefig('trend05_22.jpeg', bbox_inches="tight")

In [ ]:
# month seasonality plot

# plot accidents by month for 2017, 2018, 2019 
# requires counts in month 
sns.lineplot(data=month_counts, 
             x='month', 
             y='counts', 
             hue='year', 
             legend='full')

# add title
plt.title('Seasonal plot')

# move the legend outside of the main figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2);

plt.savefig('trend05_22_months.jpeg', bbox_inches="tight")

In [ ]:
# heatmap day and time

# split time out into hours 
accident_2005_2021['time_hour'] = accident_2005_2021.apply(lambda x: str(x.time).split(':')[0], axis=1)

# get count of hour 
d_t_table = accident_2005_2021['time_hour'].groupby([accident_2005_2021.day_of_week, accident_2005_2021.time_hour]).count()

# extra processing step to get day into format for heatmap - what is unstack?
d_t_table = d_t_table.rename_axis(['day_of_week', 'time_hour']).unstack('day_of_week')

# re index table to days of the week - does this work?
day_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
d_t_table = d_t_table.reindex(day_of_week)

# plot heatmap
sns.heatmap(data = d_t_table)
plt.savefig('trend05_22_heatmap.jpeg', bbox_inches="tight")

# use a lookup table to decipher integer replacements and use in a bar chart

In [ ]:
# read in the lookup table 
lookup_table = pd.read_excel("Road-Safety-Open-Dataset-Data-Guide.xlsx")
lookup_table.rename(columns = {'code/format':'code', 'field name':'field_name'}, inplace = True)
lookup_table

In [ ]:
# create dataframe specifically for the graphing, segment further for useful comparison bar charts
casualty_class = casualties_concat_final[['accident_index', 'casualty_class']].copy()

# create df of the appropriate look up 
class_lookup = lookup_table[['field_name', 'code', 'label']].query("field_name == 'casualty_class'")

# attach appropriate lookup so that codes are deciphered
class_w_lookup = pd.merge(casualty_class, class_lookup, how='left', left_on = 'casualty_class', right_on ='code')

# plot the data with the new label
class_w_lookup['label'].value_counts().sort_index().plot(kind="bar")


plt.xticks(rotation=45)
plt.xlabel("Casualty Class of Casualty")
plt.ylabel("Count")

plt.ylim(0, 350000)

plt.savefig('casualty_class.jpeg', bbox_inches="tight")

In [ ]:
# bar chart for counts where some counts are unpopulated

In [ ]:
# number of casualties - what is defined as a casualty?
plt.figure(figsize=(20,8))

accident_num_casualties = accidents_concat[['accident_index', 'number_of_casualties']].copy()
# accident_weather['weather_conditions'].value_counts()

# # # 
# use this to get the empty counts
# # #
accident_num_casualties = accident_num_casualties['number_of_casualties'].value_counts().\
                         reindex(range(accident_num_casualties['number_of_casualties'].min(), accident_num_casualties['number_of_casualties'].max()+1), fill_value=0)\
                         .reset_index()

ax = sns.barplot(x='index', y = 'number_of_casualties', data = accident_num_casualties)
ax.set(xlabel = 'number of casualties', ylabel = "count")

plt.xticks(rotation=45)
plt.savefig('17_19_num_cas.jpeg', bbox_inches="tight")

# plotting longitudes and latitudes

In [ ]:
import geopandas
import folium

from folium import plugins
from folium.plugins import HeatMap

In [ ]:
# Ensure you're handing it floats
acci_map = accidents_concat[["accident_index", "longitude", "latitude", "accident_severity"]].query("accident_severity == 1").copy()

acci_map['latitude'] = acci_map['latitude'].astype(float)
acci_map['longitude'] = acci_map['longitude'].astype(float)

acci_map = acci_map[['latitude', 'longitude']]
acci_map = acci_map.dropna(axis=0, subset=['latitude','longitude'])

# List of latitudes and longitude needed
acci_map = [[row['latitude'],row['longitude']] for index, row in acci_map.iterrows()]

# what is this?
# heat_data = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

# Plot it on the map
traffic_map = folium.Map(location=acci_map[0], zoom_start = 13) 
HeatMap(acci_map).add_to(traffic_map)
# traffic_map.save('cycalist_traffic_map.html')

# display the map
traffic_map